<a href="https://colab.research.google.com/github/asabbah44/Text_Classification/blob/main/Word_Emb_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.nbshare.io/notebook/197284676/Word-Embeddings-Transformers-In-SVM-Classifier-Using-Python/

In [ ]:
!pip install tensorflow_text
import tensorflow_text

In [ ]:
import numpy as np
import tensorflow_hub as hub
import tensorflow_text  # this needs to be imported to set up some stuff in the background

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
import re

import pandas as pd

In [ ]:
import string


from spacy.lang.en import stop_words as spacy_stopwords  # we use spacy's list of stop words to clean our data


stop_words = spacy_stopwords.STOP_WORDS
punctuations = string.punctuation



def clean(text):
  
    text = re.sub(r'\W+', ' ', text)  # remove non-alphanumeric characters
    # replace numbers with the word 'number'
    text = re.sub(r"\d+", "number", text)
    # don't consider sentenced with less than 3 words (i.e. assumed noise)
    if len(text.strip().split()) < 3:
        return None
    text = text.lower()  # lower case everything
    
    return text.strip() # remove redundant spaces

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/mydataset.csv')


df1 = pd.DataFrame(df1, columns = ['commenttext','label'])
df2 = pd.DataFrame(df2, columns = ['commenttext','label'])

df=df2#pd.concat([df1,df2])

df = df.assign(clean_text=df.commenttext.apply(clean)).dropna()

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
# we split the data into train and test
msg_train, msg_test, y_train, y_test = train_test_split(df.clean_text, df.label,test_size=0.2)

In [ ]:
# we just feed in the list of sentences, and we get the vector representation of each sentence
X_test = embed(msg_test)
X_test.shape

print(msg_test)

1072                 todo check for access modifiers here
595     todo come up with an intelligent way to size t...
969                            todo create security tests
599     todo a better and faster rest pattern matcher ...
1289    todo think about threading requirements for wr...
                              ...                        
179                                     unit test updates
865     testnumber src does not exceed quota and dst h...
815     todo we need revisit this test when the call b...
596     todo move uri params to a mllpconfiguration cl...
1060      todo format should come from a preference model
Name: clean_text, Length: 302, dtype: object


In [ ]:
# we don't have enough memory to apply embeddings in one shot,
# so we have to split the data into batches and concatenate them later
splits = np.array_split(msg_train, 5)
l = list()
for split in splits:
    l.append(embed(split))

In [ ]:
X_train = tf.concat(l, axis=0)
del l
X_train.shape

TensorShape([1205, 512])

In [ ]:
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
class_weight = compute_class_weight(
    class_weight='balanced', classes=["Requirement","Design","Defect","Test","Documentation"], y=y_train
)
class_weight


array([ 0.87318841,  0.53436807,  1.06167401,  1.06167401, 10.04166667])

In [ ]:
class_weight={"Requirement":class_weight[0], "Design":class_weight[1],"Defect":class_weight[2],"Test":class_weight[3],
                        "Documentation":class_weight[4]}

In [ ]:
# initialize the model and assign weights to each class
clf = SVC()
# train the model
clf.fit(X_train, y_train)
# use the model to predict the testing instances
y_pred = clf.predict(np.array(X_test))
# generate the classification report
print(classification_report(y_test, y_pred))
print('accuracy %s' % accuracy_score(y_pred, y_test))


               precision    recall  f1-score   support

       Defect       0.70      0.61      0.65        57
       Design       0.70      0.75      0.72       119
Documentation       1.00      0.14      0.25         7
  Requirement       0.56      0.64      0.59        55
         Test       0.98      0.92      0.95        64

     accuracy                           0.73       302
    macro avg       0.79      0.61      0.63       302
 weighted avg       0.74      0.73      0.72       302

accuracy 0.7251655629139073


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(np.array(X_test))
print(classification_report(y_test, y_pred))

print('accuracy %s' % accuracy_score(y_pred, y_test))
print('accuracy %s' % accuracy_score(y_pred, y_test))


               precision    recall  f1-score   support

       Defect       0.70      0.53      0.60        57
       Design       0.64      0.85      0.73       119
Documentation       0.00      0.00      0.00         7
  Requirement       0.60      0.47      0.53        55
         Test       0.96      0.86      0.91        64

     accuracy                           0.70       302
    macro avg       0.58      0.54      0.55       302
 weighted avg       0.70      0.70      0.69       302

accuracy 0.7019867549668874
accuracy 0.7019867549668874


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(np.array(X_test))
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

       Defect       0.64      0.53      0.58        57
       Design       0.68      0.76      0.72       119
Documentation       1.00      0.14      0.25         7
  Requirement       0.56      0.62      0.59        55
         Test       0.98      0.92      0.95        64

     accuracy                           0.71       302
    macro avg       0.77      0.59      0.62       302
 weighted avg       0.72      0.71      0.71       302

